# Лабораторная работа: Мета-обучение
## Шаги 5-6: Анализ мета-набора и визуализация

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
# Загрузка данных
try:
    meta_df = pd.read_csv('../data/processed/meta_features_raw.csv')
    results_df = pd.read_csv('../data/processed/algorithm_performance.csv')
    print(f"Мета-признаки: {meta_df.shape}")
    print(f"Результаты: {results_df.shape}")
except FileNotFoundError:
    print("Сначала выполните предыдущие шаги")
    meta_df = None
    results_df = None

In [ ]:
if meta_df is not None and results_df is not None:
    # Объединение
    meta_dataset = pd.merge(meta_df, results_df[['dataset_id', 'best_algorithm']], on='dataset_id')
    print(f"Мета-набор: {meta_dataset.shape}")
    print(f"Колонки: {list(meta_dataset.columns)}")

In [ ]:
if 'meta_dataset' in locals():
    # Корреляционная матрица
    feature_cols = [c for c in meta_dataset.columns 
                   if c not in ['dataset_id', 'dataset_name', 'best_algorithm']]
    corr = meta_dataset[feature_cols].corr()

    plt.figure(figsize=(14, 12))
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('Корреляционная матрица мета-признаков', fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
if 'meta_dataset' in locals():
    # Поиск высоких корреляций
    threshold = 0.95
    high_corr = []
    for i in range(len(corr.columns)):
        for j in range(i+1, len(corr.columns)):
            if abs(corr.iloc[i, j]) > threshold:
                high_corr.append({
                    'feature1': corr.columns[i],
                    'feature2': corr.columns[j],
                    'correlation': corr.iloc[i, j]
                })

    if high_corr:
        print(f"Найдено {len(high_corr)} пар с корреляцией > {threshold}:")
        for item in high_corr[:10]:  # покажем первые 10
            print(f"  {item['feature1']} — {item['feature2']}: {item['correlation']:.3f}")
    else:
        print(f"Нет пар с корреляцией выше {threshold}")

In [ ]:
if 'meta_dataset' in locals():
    # PCA
    X = meta_dataset[feature_cols].fillna(0).values
    y = meta_dataset['best_algorithm'].values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)

    plt.figure(figsize=(12, 8))
    unique_algos = np.unique(y)
    colors = plt.cm.Set1(np.linspace(0, 1, len(unique_algos)))

    for i, algo in enumerate(unique_algos):
        mask = y == algo
        plt.scatter(X_pca[mask, 0], X_pca[mask, 1], label=algo, color=colors[i], alpha=0.7, s=50)
    
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
    plt.title('PCA проекция мета-набора')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
if 'meta_dataset' in locals() and X_scaled.shape[0] > 10:
    # t-SNE
    tsne = TSNE(n_components=2, perplexity=min(30, X_scaled.shape[0]-1), random_state=42)
    X_tsne = tsne.fit_transform(X_scaled)

    plt.figure(figsize=(12, 8))
    for i, algo in enumerate(unique_algos):
        mask = y == algo
        plt.scatter(X_tsne[mask, 0], X_tsne[mask, 1], label=algo, color=colors[i], alpha=0.7, s=50)
    
    plt.xlabel('t-SNE1')
    plt.ylabel('t-SNE2')
    plt.title('t-SNE проекция мета-набора')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

### Выводы:
1. Выявлены корреляции между мета-признаками
2. PCA и t-SNE показывают разделение классов
3. Мета-набор готов для обучения